In [1]:
import pandas as pd
import geopandas as gpd

# Load Dataset

In [2]:
sat_data = pd.read_csv("no2_dataset.csv", parse_dates=["SatelliteDatetime", "SurfaceDatetime"])
sat_data = gpd.GeoDataFrame(sat_data, geometry=gpd.points_from_xy(sat_data.SurfaceLongitude, sat_data.SurfaceLatitude))

In [3]:
sat_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1622453 entries, 0 to 1622452
Data columns (total 23 columns):
 #   Column                              Non-Null Count    Dtype                               
---  ------                              --------------    -----                               
 0   AirQualityStation                   1622453 non-null  object                              
 1   SatelliteDatetime                   1622453 non-null  datetime64[ns]                      
 2   TroposphericNO2ColumnNumberDensity  1622453 non-null  float64                             
 3   SatelliteLongitude                  1622453 non-null  float64                             
 4   SatelliteLatitude                   1622453 non-null  float64                             
 5   Source                              1622453 non-null  object                              
 6   geometry                            1622453 non-null  geometry                            
 7   SurfaceCon

# Covered Countries
* The `Countrycode` column holds the country names as ISO 2-digit codes
* See e.g. https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes

In [4]:
sat_data.Countrycode.unique()

array(['DE', 'CZ', 'AT', 'SK', 'TR', 'IT', 'IE', 'FR', 'BE', 'HR', 'AL',
       'BG', 'EE', 'DK', 'CY', 'CH', 'FI', 'RO', 'SE', 'RS', 'SI', 'GR',
       'HU', 'LT', 'NL', 'MT', 'MK', 'LU', 'ME', 'LV', 'PL', 'GB', 'NO',
       'ES', 'PT'], dtype=object)

# Aggregate NO$_2$ Levels
* Compare surface and tropospheric NO2 levels by:
    * Season (winter (Dez., Jan., Feb.) vs. summer (Jun., Jul., Aug.))
    * Area (urban vs. rural)
    * Population density (above or below 500 people/km**2)
    * Lockdown response measures (StayHomeOrder vs. None)
    * Dominant pollution source (traffic vs. background)
    * Temperature (above or below 20°C)
    * Precipitation (above or below 10mm)

In [5]:
winter = [12, 1, 2]
summer = [6, 7, 8]
population_density_threshold = 500
temperature_threshold = 20
precipitation_threshold = 10

In [6]:
obs_all = sat_data
obs_winter = sat_data.loc[sat_data.Month.isin(winter)]
obs_summer = sat_data.loc[sat_data.Month.isin(summer)]
obs_urban = sat_data.loc[sat_data.AreaType == "urban"]
obs_rural = sat_data.loc[sat_data.AreaType == "rural"]
obs_pop_d_high = sat_data.loc[sat_data.PopulationDensity >= population_density_threshold]
obs_pop_d_low = sat_data.loc[sat_data.PopulationDensity < population_density_threshold]
obs_lockdown_yes = sat_data.loc[sat_data.Lockdown == "StayHomeOrder"]
obs_lockdown_no = sat_data.loc[sat_data.Lockdown == "None"]
obs_traffic = sat_data.loc[sat_data.StationType == "traffic"]
obs_background = sat_data.loc[sat_data.StationType == "background"]
obs_temp_high = sat_data.loc[sat_data.Temperature >= temperature_threshold]
obs_temp_low = sat_data.loc[sat_data.Temperature < temperature_threshold]
obs_prec_high = sat_data.loc[sat_data.Precipitation >= precipitation_threshold]
obs_prec_low = sat_data.loc[sat_data.Precipitation < precipitation_threshold]

In [7]:
subsets = {"all" : obs_all, "winter" : obs_winter, "summer" : obs_summer, "urban" : obs_urban, 
           "rural" : obs_rural, "high population density" : obs_pop_d_high, 
           "low population density" : obs_pop_d_low, "lockdown" : obs_lockdown_yes,
          "traffic" : obs_traffic, "background" : obs_background, "high temperature" : obs_temp_high,
          "low temperature" : obs_temp_low, "high precipitation" : obs_prec_high, "low precipitation" : obs_prec_low}
columns = [("","Observations"), ("Troposphere", "Mean"), ("Troposphere", "Std"), ("Surface", "Mean"), ("Surface", "Std"), ("","Count")]

In [8]:
# collect dataset statistics
statistics = []
for name, observations in subsets.items():
    data = [name]
    for i, measurement_type in enumerate(["TroposphericNO2ColumnNumberDensity", "SurfaceConcentration"]):
        relevant_obs = observations[measurement_type]
        data.extend([relevant_obs.mean(), relevant_obs.std()])
        if i == 1:
            data.extend([relevant_obs.shape[0]])
            
    statistics.append(data)
    
statistics = pd.DataFrame(data=statistics, columns=pd.MultiIndex.from_tuples(columns))

In [9]:
statistics

Troposphere              Surface             \
               Observations        Mean       Std       Mean        Std   
0                       all    0.000045  0.000040  13.747349  14.204032   
1                    winter    0.000064  0.000061  21.486494  18.375081   
2                    summer    0.000033  0.000022  10.855184  12.183615   
3                     urban    0.000048  0.000043  17.410467  15.712448   
4                     rural    0.000038  0.000033   6.346847   7.566445   
5   high population density    0.000066  0.000055  19.313643  17.325999   
6    low population density    0.000037  0.000030  11.595917  12.128650   
7                  lockdown    0.000040  0.000029   9.061628   8.720799   
8                   traffic    0.000048  0.000044  23.721059  17.788665   
9                background    0.000044  0.000039   9.807432  10.146342   
10         high temperature    0.000037  0.000025  11.822532  13.037524   
11          low temperature    0.000048  0.000045  14.482994  14.558248   
12       high precipitation    0.000036  0.000030  12.827692  13.368360   
13        low precipitation    0.000045  0.000041  13.777668  14.229739   

             
      Count  
0   1622453  
1    249058  
2    525733  
3    934757  
4    214654  
5    452283  
6   1170170  
7     87508  
8    455139  
9    966830  
10   448625  
11  1173828  
12    51782  
13  1570671

# More Details

In [10]:
sat_data.loc[:, ["AreaType", "TroposphericNO2ColumnNumberDensity", "SurfaceConcentration"]].groupby("AreaType").agg(["mean", "std", "count"])

TroposphericNO2ColumnNumberDensity                    \
                                             mean       std   count   
AreaType                                                              
rural                                    0.000038  0.000033  214654   
rural-nearcity                           0.000044  0.000035   25287   
rural-regional                           0.000033  0.000021   46025   
rural-remote                             0.000026  0.000017    8986   
suburban                                 0.000044  0.000038  392744   
urban                                    0.000048  0.000043  934757   

               SurfaceConcentration                     
                               mean        std   count  
AreaType                                                
rural                      6.346847   7.566445  214654  
rural-nearcity             7.620031   8.114452   25287  
rural-regional             4.900556   5.708919   46025  
rural-remote               2.564842   2.647838    8986  
suburban                  10.760727  11.005446  392744  
urban                     17.410467  15.712448  934757

In [11]:
sat_data.loc[:, ["StationType", "TroposphericNO2ColumnNumberDensity", "SurfaceConcentration"]].groupby("StationType").agg(["mean", "std", "count"])

TroposphericNO2ColumnNumberDensity                    \
                                          mean       std   count   
StationType                                                        
background                            0.000044  0.000039  966830   
industrial                            0.000042  0.000037  200484   
traffic                               0.000048  0.000044  455139   

            SurfaceConcentration                     
                            mean        std   count  
StationType                                          
background              9.807432  10.146342  966830  
industrial             10.105187  10.195276  200484  
traffic                23.721059  17.788665  455139

In [12]:
sat_data.loc[:, ["Lockdown", "TroposphericNO2ColumnNumberDensity", "SurfaceConcentration"]].groupby("Lockdown").agg(["mean", "std", "count"])

TroposphericNO2ColumnNumberDensity            \
                                                           mean       std   
Lockdown                                                                    
None                                                   0.000045  0.000041   
RegionalStayHomeOrder                                  0.000031  0.000031   
RegionalStayHomeOrderPartial                           0.000067  0.000064   
StayHomeGen                                            0.000048  0.000036   
StayHomeGenPartial                                     0.000044  0.000035   
StayHomeOrder                                          0.000040  0.000029   
StayHomeOrderPartial                                   0.000033  0.000022   
StayHomeRiskG                                          0.000025  0.000022   
StayHomeRiskGPartial                                   0.000039  0.000030   

                                      SurfaceConcentration                      
                                count                 mean        std    count  
Lockdown                                                                        
None                          1412130            14.211906  14.574400  1412130  
RegionalStayHomeOrder            1073            15.599942  14.246522     1073  
RegionalStayHomeOrderPartial    15580            18.462677  16.444445    15580  
StayHomeGen                     27557            11.051504  10.753840    27557  
StayHomeGenPartial              41076            10.647185  11.245395    41076  
StayHomeOrder                   87508             9.061628   8.720799    87508  
StayHomeOrderPartial            20203             9.256276  10.077292    20203  
StayHomeRiskG                   15198            12.097790  11.240592    15198  
StayHomeRiskGPartial             2128            11.870054  12.197437     2128